## What You will Learn

#### The goal of this tutorial is to introduce [Great Expectations](https://greatexpectations.io/) a python package to help ensure data quality in data science and analytics workflows. You will learn in this tutorial

1.	Why data quality is imperative for data science.
2.	What expectations are, and how you can test expectations 
3.	How to set up a great expectations suite in a production environment


## 1.Importance of Data Quality

According to a [recent survey](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/?sh=77d4a0526f63), data scientists spend around 60 percent of their time cleaning and massaging data. And this aspect of the work is often considered not very desirable.  

Data cleaning is not an aspect focused on much in coursework. In most assignments and projects, we receive already cleaned data in CSV files or access to a database and continue to build models and conduct analysis without paying too much attention to data quality issues.

The data we encounter in most real-world data science projects have issues with **accuracy, completeness, uniqueness, consistency, and timeliness**. Furthermore, we may have to work with dynamic data that may get updated daily, weekly, or monthly. Consider building a machine learning model and an analytics dashboard with [311 Data from the NYC open data portal](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9). This data gets updated daily, and we have to ensure that the columns in each new update are consistent with what we already have.

Most importantly, data quality can adversely impact model quality. For instance, when you are pre-processing a categorical variable to be entered into a model, an inconsistency between ***‘NYPD’ and ‘NyPD’*** would lead these two values to be considered as two distinct categories. But beyond this simple example, data quality issues can adversely impact model performance, and it may be often difficult to detect these issues during the modeling stage. 


## Set Up
in order to complete this section you need to install the **great_expectations** and **pandas** packages. Once you have succesfully installed them either using pip or anaconda, you can can import these packages using the following command

In [41]:
import great_expectations as ge
import pandas as pd

import scipy.stats as st
import numpy as np
import requests

##  2.1 Quick Intro to Setting Expectations

You will be working with a slightly modified version of the Titanic Dataset for this section.

Let's imagine that the Titanic was able to divert its course and dodge the iceberg that stuck it in 1912 and completed its maiden voyage. Let's assume that the RMS titanic took many voyages between Europe and the UK, and a team of data engineers and data scientists with Titanics marketing division are collecting passenger data to determine which passengers it should provide premium discounts for an all-inclusive first-class pass. And during each voyage, they receive a batch of data. 

Let's Load the titanic dataset from the github link with the `read_csv` function in great expectations

In [ ]:
titanic = ge.read_csv("https://raw.githubusercontent.com/laknath123/Practical-Data-Science-15-688-Tutorial/main/titanic.csv")

Lets Explore the first few lines of this dataset

In [ ]:
titanic.head()

In This tutorial, we hope to make some modifications to the original dataset.
-  First we we will remove the `Survived` Column
-  Next we will split this dataset into two batches
-  Create A data anamoloy by Adding the Value `Z` to the `Embarked` column and rename the `SibSp`column as `SIBSP`

In [ ]:
titanic_df = titanic.loc[:,titanic.columns !='Survived']
df_batch1= titanic_df[0:400]
df_batch2= titanic_df[400:]
df_batch2.loc[400,'Embarked']='Z' # Set an Anomolous value
df_batch2 = df_batch2.rename(columns={'SibSp':'SIBSP'}) # Rename the column to create a column name mismatch 

## 2.2 Setting Table Expectations

Table Expectations are all about standardizing the structure of your data frames. If you have worked with multiple datasets, One of the common issues you run into is errors when appending and merging two data frames. These issues are often caused by
- Not having the correct number of columns
- A mismatch between column names


In [ ]:
frames = [df_batch1,df_batch2]
pd.concat(frames).head(2)

As you can see from the above output, when we tried to concatnate the dataframes it created a new column called `SIBSP` due to the column name mismatch. Let's look at how we can leverage great expectations to detect this issue

We know that the df_batch1 dataset had the following list of columns

In [ ]:
df_batch1.columns.tolist()

We can use this list and check if df_batch2 dataframe match columns in the original list

In [ ]:
df_batch2.expect_table_columns_to_match_ordered_list(column_list=df_batch1.columns.tolist())

When you run an expectation on a dataset it returns a result like the one you see above. The most important item to look at is the success parameter. Here it is evident that `success: False` meaning that we have failed an expectation. But what's really cool is that it shows you exactly where the mismatch happened. 

Here are a some other table expectations you should explore

#### Create an Expectation on the number of rows for each of your dataset

In [ ]:
'''
since it returns a dictionary object, you can simply look at the 'success' key
You can pass in a min and max value parameter on what you expect the minimum and maximum row counts to be
'''
df_batch1.expect_table_row_count_to_be_between(min_value=350,max_value=450)['success']

#### Create an Expectation to check whether a specific column exists

In [ ]:
'''
We Want to make sure the Embarked column exists since we will use it in the next section
'''
df_batch1.expect_column_to_exist(column='Embarked')['success']

#### Test the Column count for tables

In [ ]:
print(df_batch2.expect_table_column_count_to_equal(11)['success'])
print(df_batch2.expect_table_column_count_to_equal(11)['result'])

## 2.2 Setting Expectations on Categorical Values and Exploring your Expectation_suite()

While table expectations are important to detect issues in tables and data frames, most data quality issues arise in the actual data contained within these tables itself

In [ ]:
df_batch1.Embarked.unique()

The data team knows that passengers get on board the ship from one of 3 ports. Cherbourg in France, Queenstown in Ireland, and Southhampton in the UK. If you look at the Embarked column, these ports are categorized as Cherbourg- `C`, Queenstown- `Q`, Southhampton-`S`

We can use the great expectations package to create an expectation that the Embarked column should only contain these three values

In [ ]:
df_batch1.expect_column_values_to_be_in_set('Embarked',['S','C','Q'])

Anytime you create an expectation,that expectation is stored as a config, and we can look at the config object it creates using the `get_expectation_suite()` command

In [ ]:
df_batch1.get_expectation_suite()

Since we hope to use this config to validate the data we recieve in the future, Let's save the previous expectation we created as `titanic_config` in our workspace using the following command

In [ ]:
titanic_config = df_batch1.get_expectation_suite()

In [ ]:
# You can explore the list of the expectations that you have already created using the following command
titanic_config['expectations']

Once the Data Team receives another batch of passenger data from a voyage, they can use the config they had created earlier to validate the new data.
You have to load the new batch of data and then run the validate command and pass the titanic config you created earlier as a parameter.
Load the second batch of data

In [ ]:
batch2 = df_batch2

you can now validate this second batch of data, and also pass in a parameter called `only_return_failures=True` to specifically show the validation rules that have failed

In [ ]:
batch2.validate(expectation_suite= titanic_config,only_return_failures=True)

If we look at the results key in the above output, It is evident now that we have failed our expectation since `"success": false`. 
It is also clear from this output that this expectation failed, because there was an unexpected value `Z` in the `Embarked` column

In instances where you are dealing with data that gets updated hourly, daily, weekly etc. You may want to keep track of each of your validations runs. And you you can get this information, using the following command 

In [ ]:
batch2.validate(expectation_suite= titanic_config)['meta']['run_id']

## 2.2 Expectations Around Null Values

Null values in your dataset are inevitable. In most cases, you would want to ensure that certain columns don't contain any null values. For instance, if you were hoping to use the `PassengerId` column as the Primary Key in a database table, you would want to ensure that this column is free of null values. 

Let's create an expectation to tackle this issue.

In [ ]:
expect= df_batch1.expect_column_values_to_not_be_null('PassengerId')
print(expect['result'])
print(expect['success'])

Sometimes though, you will encounter null values, but you want to ensure that you set a bound on the percentage of null values that you would tolerate. 

Let's look at the percentage of null values in our first batch of data

In [ ]:
df_batch1['Cabin'].isna().sum()
df_batch1['Cabin'].isna().sum()/len(df_batch1['Cabin'])

Based on this observation we can set an expectation that we will tolerate around 70 percent of null values in this column. The code chunk below shows that we passed the test, and that approximately 23 percent of the values in this column were null in the second dataset

In [ ]:
print(df_batch2.expect_column_values_to_be_null('Cabin',0.7)['success'])
df_batch2.expect_column_values_to_be_null('Cabin',0.7)['result']['unexpected_percent']

In [ ]:
df_batch2['Cabin'].isna().sum()/len(df_batch2['Cabin'])

## 2.4 Setting Statistical Expectations 

The great expectation package also provides ways to build expectations related to statistics from our dataset. 

Let's look at the average fare that that we charged passengers during the first voyage

In [ ]:
df_batch1['Fare'].mean()

Certain industries and organizations have [Average Cost Pricing Rules](https://www.investopedia.com/terms/a/average_cost_pricing_rule.asp) or internal policies that require them to set prices within a certain range. Let's assume that the Titanic's pricing team has determined that they want the Average Fare to fall between a confidence interval 

Let's calculate a 95 percent confidence interval for the average fare

In [ ]:
lower_b, upper_b = st.norm.interval(alpha=0.95, loc=np.mean(df_batch1['Fare']), scale=st.sem(df_batch1['Fare']))
print(lower_b)
print(upper_b)

We can set an expectation to test this using the `expect_column_mean_to_be_between` method and run this expectation on our second batch of data with the upper and lower bounds we calculated in the earlier step

In [ ]:
df_batch2.expect_column_mean_to_be_between('Fare',lower_b,upper_b)

When we run this expectation on our second batch of data, it is evident that the average fare falls within the expected range

We can also set an expectation for the median value of columns and provide a range of values that we would tolerate. 
We can use expectations to identify outliers that fall outside of the  interquartile range or outside the 25th and 75th percentiles of an array

In [ ]:
df_batch1['Fare'].median()

In [ ]:
perce_25th = np.percentile(df_batch1['Fare'],25)
perce_75th = np.percentile(df_batch1['Fare'],75)
print("25th percentile of the dataset is ",perce_25th)
print("75th percentile of the dataset is ",perce_75th)

Let's set an expectation to see if any of the subsequent batches of data have a median value that is outside of this range for the `Fare` column


In [ ]:
print(df_batch2.expect_column_median_to_be_between('Fare',perce_25th,perce_75th)['result'])
print(df_batch2.expect_column_median_to_be_between('Fare',perce_25th,perce_75th)['success'])

## 3.0 Setting up an Expectation Suite for the Yelp Api

Let's put everything that we learned so far together and 

In [ ]:
data_dict = response.json()
data_source= pd.json_normalize(data_dict['businesses'])
data_source.head()
data_source.to_csv('yelp_batch.csv')

In [25]:
batch = ge.read_csv('yelp_batch.csv')

In [26]:
batch.head(2)

,Unnamed: 0,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,0,L4r6klm7FG4QBbNe8gu5-A,commonplace-coffee-pittsburgh-3,Commonplace Coffee,https://s3-media2.fl.yelpcdn.com/bphoto/5TYkkK...,False,https://www.yelp.com/biz/commonplace-coffee-pi...,207,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,...,40.438236,-79.926516,5827 Forbes Ave,NaN,NaN,Pittsburgh,15217,US,PA,"['5827 Forbes Ave', 'Pittsburgh, PA 15217']"
1,1,DPR86ATfmOo3s9kDhQrzzA,round-table-coffee-pittsburgh,Round Table Coffee,https://s3-media1.fl.yelpcdn.com/bphoto/e8-XBa...,False,https://www.yelp.com/biz/round-table-coffee-pi...,16,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",5.0,...,40.456334,-79.930275,5830 Ellsworth Ave,Ste 100,NaN,Pittsburgh,15232,US,PA,"['5830 Ellsworth Ave', 'Ste 100', 'Pittsburgh,..."


In [23]:
set(batch.is_closed)

{False}

In [29]:
columm_list = batch.columns.tolist()
# Lets set the following expectations
batch.expect_table_columns_to_match_ordered_list(columm_list)
batch.expect_column_values_to_not_be_null('id')
batch.expect_column_max_to_be_between('rating',5.0)
batch.expect_column_mean_to_be_between('rating',4.4,5.0) # Average coffee shop rating to be between 4.0 and 5.0
lower_b, upper_b = st.norm.interval(alpha=0.95, loc=np.mean(batch['review_count']), scale=st.sem(batch['review_count']))
print(lower_b)
print(upper_b)
batch.expect_column_mean_to_be_between('review_count',lower_b,upper_b)

60.96446788208796
109.15553211791205


{
  "meta": {},
  "success": true,
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 85.06,
    "element_count": 50,
    "missing_count": null,
    "missing_percent": null
  }
}

In [30]:
yelp_config = batch.get_expectation_suite()

In [31]:
yelp_config['expectation_suite_name'] = 'yelp_expectations'
yelp_config['data_asset_type']='pandas dataframe'

In [32]:
yelp_config

{
  "meta": {
    "great_expectations_version": "0.14.10"
  },
  "expectation_suite_name": "yelp_expectations",
  "data_asset_type": "pandas dataframe",
  "ge_cloud_id": null,
  "expectations": [
    {
      "meta": {},
      "kwargs": {
        "column_list": [
          "Unnamed: 0",
          "id",
          "alias",
          "name",
          "image_url",
          "is_closed",
          "url",
          "review_count",
          "categories",
          "rating",
          "transactions",
          "price",
          "phone",
          "display_phone",
          "distance",
          "coordinates.latitude",
          "coordinates.longitude",
          "location.address1",
          "location.address2",
          "location.address3",
          "location.city",
          "location.zip_code",
          "location.country",
          "location.state",
          "location.display_address"
        ]
      },
      "expectation_type": "expect_table_columns_to_match_ordered_list"
    },
  

In [44]:
api_key = 'yFPQl70OTPmsWvi2XU5lgIeMUVfKGrMx7Jj5lPnvxgX_DJLoX4tRfIct7GgALvkgvFNeMOhrHTU0v5p4pvRH2qgWu8IkhaRZQ68LdjVmK9jh4sPdpR_NU0xA_0X5YXYx'

In [50]:
def get_request(term, location):
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    search_api_url = 'https://api.yelp.com/v3/businesses/search'
    params = {'term': term, 
          'location': location,
          'limit': 50}
    response = requests.get(search_api_url, headers=headers, params=params, timeout=5)
    data_dict = response.json()
    data_source= pd.json_normalize(data_dict['businesses'])
    data_source.head()
    data_source.to_csv('newbatch.csv')

In [51]:
get_request('coffee','Shadyside, Pittsburgh')

In [52]:
new_batch = ge.read_csv('newbatch.csv')

In [55]:
validation = new_batch.validate(expectation_suite= yelp_config,only_return_failures=True)

In [57]:
validation['statistics']

{'evaluated_expectations': 5,
 'successful_expectations': 3,
 'unsuccessful_expectations': 2,
 'success_percent': 60.0}

## 2.4 Explore The World of Expectations

We just scratched the surface of what great expectations have to offer in the previous section. 
The great expectations library has a host of tests to check issues with your data. These test include ways to evaluate statistical expectations, expectations related to regular expressions, and ones to deal with geospatial such as setting latitude and longitude values to be within a certain range.

You can check out the entire list of expectations available using the this [link](https://greatexpectations.io/expectations)

Since Great Expectations is an open-source package, the community can contribute new expectations, and according to the website the goal of this package is create a **SHARED, OPEN STANDARD OF DATA QUALITY**

### 3.0 Setting up a Great Expectation suite in a production environment (Optional Section)

The data science workflow in a typical organization involves obtaining data from many sources. For instance, the data may come from enterprise resource planning systems, customer relationship management systems, or from sensors.

In an analytics workflow that spans multiple teams, data engineering teams create pipelines to move data and may create databases and data warehouses to store data. These pipelines would then be used by data analysts/ business intelligence developers to create dashboards, and data scientists to build and deploy machine learning models.
 
Great Expectations seems to be a popular technology  currently being used by data engineers and data science teams to run data quality tests before  ingesting data into databases and models

- [How Great Expectations is used by the Global Analytics Team at Heineken](https://greatexpectations.io/case-studies/heineken-case-study/)

 
In this section, I guide you through how great_expectations may be deployed in a typical data science workflow where you have to deal with dynamic data inflows as opposed to dealing with a CSV file at a time.
 
Again, a real-world workflow may be a lot more involved than what I present here, but I hope that this gives you a general idea
 

## Set up
1.	Download the zipfile or obtain it from github, and save it somewhere in your machine.
2.	Open the anaconda prompt or terminal and use `cd` to go to the directory where the file is saved
3.	once you have cd’d into that folder. Create a virtual environment with the following command `python -m venv venv`
4.	Activate this virtual environment by running `venv\Scripts\activate.bat`
5.	Once you have activated the virtual environment run. 
    
    -`pip install great_expectations`  
    -`pip install sqlalchemy`


## 3.1 Launch and initialize Great Expectations
 
Run the following command

`Great_expectations init`

The command above initializes Great Expectations in your folder, and if you look at the folder, you should be able to see that you have a new folder named great_expectations, and it has the structure described in the following image